### Use this notebook for developing mlframe prototype 2

In [1]:
from commands import commands
commands.lscom()

Scripts location: /Users/peterbryzgalov/work/ML/mlframework/mlframe/commands/../scripts
execremote 		Execute command on remote machine
test_nvidia_ubuntu 		Get versions of installed Nvidia drivers and CUDA
test_docker_ubuntu 		Get versions of Docker and nvidia-docker
install_docker_ubuntu 		Install Docker on Ubuntu
install_nvdocker_ubuntu 		Install nvidia-docker on Ubuntu
test 		Test script with sleep


In [2]:
commands.remote("DLbox","test")

Command file name is /Users/peterbryzgalov/work/ML/mlframework/mlframe/commands/../scripts/test.sh


In [5]:
commands.execremote("DLbox","test.sh")

Calling exec_remote.sh ('DLbox', 'test.sh')
REMOTE_ADDRESS=DLbox
SCRIPT_FILE_PATH=./test.sh
PWD: /Users/peterbryzgalov/work/ML/mlframework/mlframe/scripts
REMOTE_COMMAND=test.sh
exec_remote.sh arguments: 
Run ./test.sh  on DLbox
Running test command on dl-box
dl-box:1
dl-box:2
dl-box:3
dl-box:4
dl-box:5
dl-box:6
Exit code 0


In [4]:
commands.execremote("DLbox","test_nvidia_ubuntu.sh")

Calling exec_remote.sh ('DLbox', 'test_nvidia_ubuntu.sh')
REMOTE_ADDRESS=DLbox
SCRIPT_FILE_PATH=./test_nvidia_ubuntu.sh
PWD: /Users/peterbryzgalov/work/ML/mlframework/mlframe/scripts
REMOTE_COMMAND=test_nvidia_ubuntu.sh
exec_remote.sh arguments: 
Run ./test_nvidia_ubuntu.sh  on DLbox
remote_command_.sh arguments: 
cuDNN not installed
NVDRV:384.90,CUDA:8.0,cuDNN:x
Exit code 0


In [38]:
import multiprocessing
import subprocess
import os

In [48]:
class BashExecutor(multiprocessing.Process):
    debug = True
    def __init__(self,command,*args):
        self.command=command
        self.args=args
        #self.daemon = True
        
    def run(self):
        print "In {}".format(self.name)
        if self.debug:
            print "Calling",command,args

        proc = subprocess.Popen(self.command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
        self.__handleOutput__(proc)
        print "Exiting",multiprocessing.current_process().name

    def __handleOutput__(self,proc):
        for line in iter(proc.stdout.readline, b''):
            print line,

In [49]:
def RemoteExec(hosts,command,*args):
    package_directory = os.path.dirname(os.getcwd())
    scripts_location=os.path.realpath(os.path.join(package_directory,"..","scripts"))
    exec_remote_script="exec_remote.sh"    
    if os.path.isfile(scripts_location):
        exe_command=[scripts_location]
    else:
        exe_command=[command]
          
    procs = []
    if type(hosts) is str:
        hosts = [hosts]
    for host in hosts:
        print host
        d = BashExecutor(exe_command, args)
        procs.append(d)
        
        d.start()
        d.join()
    
    print "Exec_command",exe_command

In [50]:
RemoteExec("mouse","test.sh -a -b")

mouse


AttributeError: 'BashExecutor' object has no attribute '_popen'